In [1]:
import pandas as pd
import numpy as np
import datetime
from pandas_summary import DataFrameSummary

In [2]:
df = pd.read_feather('train_normalized_data.fth')
df_test = pd.read_feather('test_normalized_data.fth')

In [3]:
df_train = df[df.Date < datetime.datetime(2015, 7, 1)]  
df_val = df[df.Date >= datetime.datetime(2015, 7, 1)]

In [4]:
final_train = False

In [5]:
def get_metric(sales, sales_):
    return (((sales - sales_)/sales)**2).sum()/len(sales)


$\textrm{RMSE} = \sqrt{\frac{1}{n} \sum_{i=1}^{n} \left(\frac{\hat{y}_i - y_i}{y_i}\right)^2}$


In [42]:
df_train.columns

Index(['index', 'Store', 'DayOfWeek', 'Date', 'Sales', 'Customers', 'Open',
       'Promo', 'StateHoliday', 'SchoolHoliday', 'Year', 'Month', 'Week',
       'Day', 'StoreType', 'Assortment', 'CompetitionDistance',
       'CompetitionOpenSinceMonth', 'CompetitionOpenSinceYear', 'Promo2',
       'Promo2SinceWeek', 'Promo2SinceYear', 'PromoInterval',
       'CompetitionOpenSince', 'Promo2Since', 'State', 'file', 'week', 'trend',
       'Date_y', 'Month_y', 'Day_y', 'file_DE', 'week_DE', 'trend_DE',
       'Date_DE', 'State_DE', 'Month_DE', 'Day_DE', 'file_y',
       'Max_TemperatureC', 'Mean_TemperatureC', 'Min_TemperatureC',
       'Dew_PointC', 'MeanDew_PointC', 'Min_DewpointC', 'Max_Humidity',
       'Mean_Humidity', 'Min_Humidity', 'Max_Sea_Level_PressurehPa',
       'Mean_Sea_Level_PressurehPa', 'Min_Sea_Level_PressurehPa',
       'Max_VisibilityKm', 'Mean_VisibilityKm', 'Min_VisibilitykM',
       'Max_Wind_SpeedKm_h', 'Mean_Wind_SpeedKm_h', 'Max_Gust_SpeedKm_h',
       'Precipitatio

In [71]:
df_train['Events']

30188     20
30189     20
30190      0
30191     20
30192     20
          ..
844333    10
844334    10
844335    10
844336     1
844337    10
Name: Events, Length: 814150, dtype: int64

In [6]:
max_sales = df_train['Sales'].max()
df.loc[:, 'Sales_norm'] = df['Sales'].values/max_sales

In [7]:
df_train.loc[:, 'Sales_norm'] = df_train['Sales'].values/max_sales
df_val.loc[:, 'Sales_norm'] = df_val['Sales'].values/max_sales

/home/usuario/anaconda3/envs/tensorflow2/lib/python3.6/site-packages/pandas/core/indexing.py:845: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/home/usuario/anaconda3/envs/tensorflow2/lib/python3.6/site-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [8]:
print('Train:')
print(get_metric(df_train['Sales_norm'], 
                 df_train['Sales_norm'].mean()))
print('Val:')
get_metric(df_val['Sales_norm'], 
           df_train['Sales_norm'].mean())

Train:
0.4059077987085825
Val:


0.30213969475255925

In [91]:
def get_mean_by_column(column, sales_str):
    group_means_dict = {}
    group_mean_list = []
    for col_value, group_df in df_train.groupby(column):
        group_mean =  group_df[group_df[sales_str] > 0][sales_str].mean()
        group_means_dict[col_value] = group_mean
        group_mean_list.append(group_mean)
    print('Train:', get_metric(df_train[sales_str], 
                               df_train[column].apply(group_means_dict.get)))
    print('Val:', get_metric(df_val[sales_str], 
                             df_val[column].apply(group_means_dict.get)))
    return group_means_dict, group_mean_list

In [92]:
# Media por store
_ = get_mean_by_column('Store', 'Sales_norm')

Train: 0.15688835922756375
Val: 0.09435624012794681


In [93]:
# Media por dia de la semana
_ = get_mean_by_column('DayOfWeek', 'Sales_norm')

Train: 0.36106577837306253
Val: 0.2561769086693393


In [51]:
# Media por numera de semana (1-52)
_ = get_mean_by_column('Week', 'Sales_norm')

Train: 0.3693486112676764
Val: 0.2674408728623329


In [50]:
_ = get_mean_by_column('Month', 'Sales_norm')

Train: 0.4002726878411708
Val: 0.2964421798587632


In [80]:
_ = get_mean_by_column('StateHoliday', 'Sales_norm')

Train: 0.4070020265579082
Val: 0.30186165673507453


In [81]:
_ = get_mean_by_column('SchoolHoliday', 'Sales_norm')

Train: 0.4034957752536508
Val: 0.3050227258443403


In [12]:
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Embedding, Input, Flatten, Concatenate, Dense, BatchNormalization, Activation, LeakyReLU, Dropout
from tensorflow.keras.regularizers import l2
from tensorflow.keras import optimizers
from tensorflow.keras import backend as K
from tensorflow.keras.utils import to_categorical

In [13]:
def rmspe(y_true, y_pred):
    return K.mean(K.square((y_true - y_pred)/y_true))

In [134]:
def get_keras_LR(X_columns):
    inputs = []
    for i, col in enumerate(X_columns):
        inp = Input(shape=(X_train[i].shape[1],), name=f"{col}_input")
        inputs.append(inp)
    if len(X_columns)>1:
        concat_out = Concatenate()(inputs)
        dense_out = Dense(1, name='Dense')(concat_out)
    else:
        dense_out = Dense(1, name='Dense')(inputs[0])
    model = Model(inputs, dense_out)
    model.compile(optimizers.Adam(lr=0.0001), loss='mse', metrics=[rmspe, 'mse'])
    return model


def get_embedings_NN(X_columns, hidden_units = 20, activation = 'relu'):
    embed_outs = []
    inputs = []
    for i, col in enumerate(X_columns):
        inp = Input(shape=(1,), name=f"{col}_input")
        inputs.append(inp)
        embed_out = Embedding(len(np.unique(X_train[i])), embed_outs_dict[col], name=f"{col}_embedding", mask_zero=False)(inp)
        out = Flatten(name=f"{col}_flatten")(embed_out)
        embed_outs.append(out)
    if len(X_columns)>1:
        concat_out = Concatenate()(embed_outs)
        dense_out = Dense(hidden_units, activation=activation)(concat_out)
    else:
        dense_out = Dense(hidden_units, activation=activation)(out)
    out = Dense(1)(dense_out)
    model = Model(inputs, out)
    model.compile(optimizers.Adam(lr=0.0001), loss='mse', metrics=[rmspe, 'mse'])
    return model

In [135]:
log_reg = True

embed_outs_dict = {'Store': 50, 'DayOfWeek': 2, 'Week': 10, 'Month': 4}
embed_outs_dict = {'Store': 50}
X_columns = list(embed_outs_dict.keys())

if final_train:
    X_train = np.hsplit(df[X_columns].values, len(X_columns))
    y_train = df['Sales_norm']
else:
    X_train = np.hsplit(df_train[X_columns].values, len(X_columns))
    y_train = df_train['Sales_norm']
    
X_val = np.hsplit(df_val[X_columns].values, len(X_columns))
X_test = np.hsplit(df_test[X_columns].values, len(X_columns))

if log_reg:
    for i in range(len(X_train)):
        X_train[i] = to_categorical(X_train[i])
        X_val[i] = to_categorical(X_val[i])
        X_test[i] = to_categorical(X_test[i])

y_val = df_val['Sales_norm']

In [140]:
if not log_reg:
    model = get_embedings_NN(X_columns)
else:
    model = get_keras_LR(X_columns)
model.summary()

Model: "model_22"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Store_input (InputLayer)     [(None, 1115)]            0         
_________________________________________________________________
dense_36 (Dense)             (None, 1)                 1116      
Total params: 1,116
Trainable params: 1,116
Non-trainable params: 0
_________________________________________________________________


In [141]:
# weights = model.get_weights()
# weights[0] = np.array(np.array(stores_mean_list).reshape(-1, 1))
# model.set_weights(weights)

In [142]:
model.evaluate(X_val, y_val)

944/944 [==============================] - 1s 721us/step - loss: 0.0359 - rmspe: 1.1011 - mse: 0.0359


[0.03587775677442551, 1.1010745763778687, 0.03587775677442551]

In [143]:
epochs = 5
if final_train:
    model.fit(X_train, y_train, epochs=epochs)
else:
    model.fit(X_train, y_train, epochs=epochs, validation_data=(X_val, y_val))

Epoch 1/5
25443/25443 [==============================] - 22s 865us/step - loss: 0.0038 - rmspe: 0.2067 - mse: 0.0038 - val_loss: 0.0018 - val_rmspe: 0.0960 - val_mse: 0.0018
Epoch 2/5
25443/25443 [==============================] - 22s 864us/step - loss: 0.0023 - rmspe: 0.1580 - mse: 0.0023 - val_loss: 0.0018 - val_rmspe: 0.0942 - val_mse: 0.0018
Epoch 3/5
25443/25443 [==============================] - 22s 871us/step - loss: 0.0023 - rmspe: 0.1569 - mse: 0.0023 - val_loss: 0.0017 - val_rmspe: 0.0948 - val_mse: 0.0017
Epoch 4/5
25443/25443 [==============================] - 22s 866us/step - loss: 0.0023 - rmspe: 0.1583 - mse: 0.0023 - val_loss: 0.0017 - val_rmspe: 0.0943 - val_mse: 0.0017
Epoch 5/5
25443/25443 [==============================] - 22s 870us/step - loss: 0.0023 - rmspe: 0.1567 - mse: 0.0023 - val_loss: 0.0018 - val_rmspe: 0.0944 - val_mse: 0.0018


In [ ]:
model.evaluate(X_train, y_train)

In [144]:
model.evaluate(X_val, y_val)

944/944 [==============================] - 1s 716us/step - loss: 0.0018 - rmspe: 0.0944 - mse: 0.0018


[0.0017532232450321317, 0.09439808875322342, 0.0017532232450321317]

In [59]:
train_predictions = model.predict(X_train)*max_sales
get_metric(df_train['Sales'].values, train_predictions.reshape(-1))

0.06149980579432537

In [60]:
test_predictions = model.predict(X_test)*max_sales
test_predictions[df_test['Open'] == 0] = 0

In [61]:
sample_csv = pd.read_csv('dataset/rossmann/sample_submission.csv')
sample_csv['Sales'] = test_predictions
sample_csv.head()

sample_csv.to_csv(f'submision_baseline_{"-".join(X_columns)}.csv', index=False)
